# Sudoku (Human Agent)
Enter numbers 1-9 into the empty cells, ensuring each row, column, and 3x3 box only contains a single instance of each digit.

You can select puzzles by difficulty level and puzzle number using the dropdowns below.

In [227]:
from IPython.display import display, HTML, Javascript, clear_output
import ipywidgets as widgets
import json
import sys
import os

# Add the scripts directory to Python's path so we can import from it
scripts_dir = os.path.join(os.getcwd(), 'scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Import puzzles from SudokuPuzzles.py
from SudokuPuzzles import PATTERNED, DENSE_RANDOM, SPARSE_RANDOM

EASY_LABEL = 'Easy (Patterned)'
MEDIUM_LABEL = 'Medium (Dense)'
HARD_LABEL = 'Hard (Sparse)'

puzzle_collections = {
    EASY_LABEL: PATTERNED,
    MEDIUM_LABEL: DENSE_RANDOM,
    HARD_LABEL: SPARSE_RANDOM
}

def convert_2d_to_1d(grid_2d):
    return [cell for row in grid_2d for cell in row]

# Current selected sudoku grid - initialize with first easy puzzle
sudoku_grid = convert_2d_to_1d(PATTERNED[1])

# Create dropdown widgets for puzzle selection
difficulty_dropdown = widgets.Dropdown(
    options=list(puzzle_collections.keys()),
    value=EASY_LABEL,
    description='Difficulty:',
    disabled=False,
)

# Default puzzle numbers for each difficulty
puzzle_numbers = {}
for difficulty in puzzle_collections:
    puzzle_numbers[difficulty] = list(puzzle_collections[difficulty].keys())

puzzle_dropdown = widgets.Dropdown(
    options=puzzle_numbers[EASY_LABEL],
    value=1,
    description='Puzzle #:',
    disabled=False,
)

# Updates puzzle number dropdown based on difficulty selection
def update_puzzle_dropdown(*args):
    difficulty = difficulty_dropdown.value
    puzzle_dropdown.options = puzzle_numbers[difficulty]
    puzzle_dropdown.value = puzzle_numbers[difficulty][0]
    # Update the main puzzle when the difficulty changes
    update_puzzle()

# Updates the displayed puzzle
def update_puzzle(*args):
    global sudoku_grid
    difficulty = difficulty_dropdown.value
    puzzle_num = puzzle_dropdown.value

    # Get the 2D grid and convert to 1D for display
    grid_2d = puzzle_collections[difficulty][puzzle_num]
    sudoku_grid = convert_2d_to_1d(grid_2d)

    draw_sudoku_grid()

# Register callbacks for selects
difficulty_dropdown.observe(update_puzzle_dropdown, names='value')
puzzle_dropdown.observe(update_puzzle, names='value')

# Generate the HTML for the Sudoku grid
def generate_sudoku_html():
    html = '''
    <style>
    .sudoku-table input {
        width: 30px;
        height: 30px;
        text-align: center;
        font-size: 16px;
        border: 1px solid lightgray;
    }
    .sudoku-table td {
        padding: 0;
    }
    .sudoku-table {
        border-collapse: collapse;
    }
    .sudoku-table td:nth-child(3),
    .sudoku-table td:nth-child(6) {
        border-right: 2px solid black;
    }
    .sudoku-table tr:nth-child(3) td,
    .sudoku-table tr:nth-child(6) td {
        border-bottom: 2px solid black;
    }
    .sudoku-table td:first-child {
        border-left: 2px solid black;
    }
    .sudoku-table tr:first-child td {
        border-top: 2px solid black;
    }
    .sudoku-table tr:last-child td {
        border-bottom: 2px solid black;
    }
    .sudoku-table td:last-child {
        border-right: 2px solid black;
    }
    </style>
    <table class="sudoku-table">
    '''

    for r in range(9):
        html += '<tr>'
        for c in range(9):
            val = sudoku_grid[r * 9 + c]
            value_attr = f'value="{val}" disabled' if val != 0 else ''
            html += f'<td><input id="cell-{r}-{c}" type="text" maxlength="1" {value_attr}></td>'
        html += '</tr>'
    html += '</table>'
    return html

# Display area for the Sudoku grid
grid_output = widgets.Output()

# Draw/redraw the Sudoku grid
def draw_sudoku_grid():
    with grid_output:
        clear_output(wait=True)
        display(HTML(generate_sudoku_html()))

# Display puzzle selection widgets
selection_widgets = widgets.HBox([difficulty_dropdown, puzzle_dropdown])
display(selection_widgets)
display(grid_output)

# Draw the initial grid
draw_sudoku_grid()

# Hidden Textarea for grid sync (communicates between Python and JS)
data_holder = widgets.Textarea(value="[]", layout={'display': 'none'})
check_button = widgets.Button(description="Check Sudoku")
reset_button = widgets.Button(description="Reset Puzzle")
output = widgets.Output()
shared = {'answers': sudoku_grid}

def handle_data_holder_change(change):
    with output:
        try:
            shared['answers'] = [int(x) for row in json.loads(change['new']) for x in row]
        except Exception as e:
            print("Failed to parse grid:", e)

data_holder.observe(handle_data_holder_change, names='value')

def is_valid_sudoku(grid):
    # Check that grid is complete (no zeros)
    if 0 in grid:
        return False

    # Validate rows and columns.
    for i in range(9):
        row = set()
        column = set()
        for j in range(9):
            # Validate rows
            row_cell = grid[i * 9 + j]
            if row_cell in row or row_cell == 0:
                return False
            row.add(row_cell) if row_cell != 0 else None

            # Validate columns
            column_cell = grid[j * 9 + i]
            if column_cell in column or column_cell == 0:
                return False
            column.add(column_cell) if column_cell != 0 else None

    # Validate 3x3 squares
    for block_i in range(3):
        for block_j in range(3):
            square = set()
            for i in range(3):
                for j in range(3):
                    index = (block_i * 3 + i) * 9 + (block_j * 3 + j)
                    if grid[index] in square or grid[index] == 0:
                        return False
                    square.add(
                        grid[index]) if grid[index] != 0 else None

    return True

def on_check_clicked(b):
    with output:
        clear_output()
        try:
            grid = shared['answers']
            if not (isinstance(grid, list) and len(grid) == 81):
                print("Grid format is invalid")
                return
            if is_valid_sudoku(grid):
                print("Valid solution")
            else:
                print("Invalid solution")
        except Exception as e:
            print("Error parsing grid:", e)

def on_reset_clicked(b):
    update_puzzle()
    with output:
        clear_output()
        print("Puzzle has been reset")

check_button.on_click(on_check_clicked)
reset_button.on_click(on_reset_clicked)

# Display buttons and output area
button_area = widgets.HBox([check_button, reset_button])
display(data_holder, button_area, output)

display(Javascript("""
(() => {
    function updateDataHolder() {
        const grid = [];
        for (let r = 0; r < 9; r++) {
            const row = [];
            for (let c = 0; c < 9; c++) {
                const el = document.getElementById(`cell-${r}-${c}`);
                let val = el?.value.trim();
                let num = parseInt(val);
                if (isNaN(num)) num = 0;
                row.push(num);
            }
            grid.push(row);
        }

        const textArea = [...document.querySelectorAll('.widget-textarea textarea')][0];
        if (textArea) {
            textArea.value = JSON.stringify(grid);
            textArea.dispatchEvent(new Event("change", { bubbles: true }));
        } else {
            console.error("Couldn't find textarea to sync data.");
        }
    }

    // Update when check button is clicked
    const checkButton = [...document.querySelectorAll('button')].find(b => b.textContent.includes("Check Sudoku"));
    if (checkButton) {
        checkButton.addEventListener("click", updateDataHolder, { once: false });
    }

    // Also update when any cell changes
    document.addEventListener('input', (event) => {
        if (event.target.id && event.target.id.startsWith('cell-')) {
            updateDataHolder();
        }
    });
})();
"""))


Output()

Textarea(value='[]', layout=Layout(display='none'))

Output()

<IPython.core.display.Javascript object>